# Reviews

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score

# ------------------
# Load the data from the source
# ------------------

reviews_dataset = pd.read_csv("./amazon_review_ID.shuf.lrn.csv");

### Preprocessing

In [2]:
# -> Since the dataset contains no missing (incl. null) values, no preprocessing in this regard is necessary
print(reviews_dataset.isnull().sum().sum());

# -> Since all the feature values are already numeric, no transformation is necessary

# Split into features and labels
reviews_dataset_features = reviews_dataset.drop(columns=["ID", "Class"]);
reviews_dataset_labels = reviews_dataset["Class"];

0


### Data splits

We now define the three data splits used for e.g. hyperparameter tuning and holdout evaluation.

In [3]:
def create_split(features, labels, test_size, rval=18):
  return train_test_split(features, labels, test_size=test_size, random_state=rval, stratify=labels);

reviews_x_train_90, reviews_x_test_90, reviews_y_train_90, reviews_y_test_90 = create_split(reviews_dataset_features, reviews_dataset_labels, 0.1);
reviews_x_train_70, reviews_x_test_70, reviews_y_train_70, reviews_y_test_70 = create_split(reviews_dataset_features, reviews_dataset_labels, 0.3);
reviews_x_train_50, reviews_x_test_50, reviews_y_train_50, reviews_y_test_50 = create_split(reviews_dataset_features, reviews_dataset_labels, 0.5);

### Train baseline models

Use cross-validation for hyperparameter tuning.

In [4]:
reviews_test_hyper_params_settings = {
    "model__n_estimators": [50, 100, 200, 400], # Nr. of decision trees
    "model__max_depth": [5, 10], # Max tree depth
    "model__max_features": ["sqrt", "log2"], # Max number of features to consider when searching for best split
    "model__criterion": ["gini", "entropy"],
};

def compute_base_line_model(x_train, y_train, cv=5):
  reviews_rf_pipeline = Pipeline([
    ("model", RandomForestClassifier(random_state=18)),
  ]);

  rf_baseline_grid = GridSearchCV(
    estimator=reviews_rf_pipeline,
    param_grid=reviews_test_hyper_params_settings,
    cv=cv,
    n_jobs=-1,
    scoring="accuracy",
  );

  return rf_baseline_grid.fit(x_train, y_train);

reviews_baseline_model_50_split = compute_base_line_model(reviews_x_train_50, reviews_y_train_50);
print(f"50/50 split - best performing hyperparameters based on accuracy: {reviews_baseline_model_50_split.best_params_}");
print(f"Best Score: {reviews_baseline_model_50_split.best_score_}\n");

reviews_baseline_model_70_split = compute_base_line_model(reviews_x_train_70, reviews_y_train_70);
print(f"70/30 split - best performing hyperparameters based on accuracy: {reviews_baseline_model_70_split.best_params_}");
print(f"Best Score: {reviews_baseline_model_70_split.best_score_}\n");

reviews_baseline_model_90_split = compute_base_line_model(reviews_x_train_90, reviews_y_train_90);
print(f"90/10 split - best performing hyperparameters based on accuracy: {reviews_baseline_model_90_split.best_params_}");
print(f"Best Score: {reviews_baseline_model_90_split.best_score_}\n");

reviews_baseline_model_cv_5 = compute_base_line_model(reviews_dataset_features, reviews_dataset_labels);
print(f"cv 5 - best performing hyperparameters based on accuracy: {reviews_baseline_model_cv_5.best_params_}");
print(f"Best Score: {reviews_baseline_model_cv_5.best_score_}\n");

reviews_baseline_model_cv_10 = compute_base_line_model(reviews_dataset_features, reviews_dataset_labels, cv=10);
print(f"cv 10 - best performing hyperparameters based on accuracy: {reviews_baseline_model_cv_10.best_params_}");
print(f"Best Score: {reviews_baseline_model_cv_10.best_score_}\n");

50/50 split - best performing hyper-parameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 400}
Best Score: 0.4293333333333334

70/30 split - best performing hyper-parameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 400}
Best Score: 0.5028571428571429

90/10 split - best performing hyper-parameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 400}
Best Score: 0.597037037037037

cv 5 - best performing hyper-parameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 400}
Best Score: 0.6333333333333334

cv 10 - best performing hyper-parameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 4

##### Holdout and Cross-Validation Evaluation

We now use the found hyperparameters to evaluate our models via holdout and cross-validation.

In [5]:
def reviews_eval_model(model, x_test, y_test, split_name):
  print(f"---------- Start eval {split_name}-------------------");
  print(f"Accuracy: {accuracy_score(model.predict(x_test), y_test)}");
  print(f"Precision: {precision_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"Recall: {recall_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"F1-Score: {f1_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"---------- End eval {split_name} -------------------\n");

# Perform holdout evaluation on the different data splits

reviews_eval_model(reviews_baseline_model_50_split, reviews_x_test_50, reviews_y_test_50, "50/50 split");
reviews_eval_model(reviews_baseline_model_70_split, reviews_x_test_70, reviews_y_test_70, "70/30 split");
reviews_eval_model(reviews_baseline_model_90_split, reviews_x_test_90, reviews_y_test_90, "90/10 split");

# Perform cross-validation for fold size 5 and 10

reviews_scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

reviews_rf_pipeline_cv = Pipeline([
    ("model", RandomForestClassifier(random_state=18, n_estimators=400, criterion="gini", max_depth=10, max_features="sqrt")),
]);

review_cv_5_results = cross_validate(reviews_rf_pipeline_cv, reviews_dataset_features, reviews_dataset_labels, cv=5, scoring=reviews_scoring)
review_cv_10_results = cross_validate(reviews_rf_pipeline_cv, reviews_dataset_features, reviews_dataset_labels, cv=10, scoring=reviews_scoring)

print("CV-5 results");
for metric in reviews_scoring:
    print(f"{metric}: {review_cv_5_results['test_' + metric].mean():.4f}")

print("CV-10 results");
for metric in reviews_scoring:
    print(f"{metric}: {review_cv_10_results['test_' + metric].mean():.4f}")

---------- Start eval 50/50 split-------------------
Accurracy: 0.5146666666666667
Precision: 0.4995634920634921
Recall: 0.5999117924559101
F1-Score: 0.4828622142660681
---------- End eval 50/50 split -------------------

---------- Start eval 70/30 split-------------------
Accurracy: 0.6622222222222223
Precision: 0.6366666666666666
Recall: 0.6944051226551227
F1-Score: 0.617660992602169
---------- End eval 70/30 split -------------------

---------- Start eval 90/10 split-------------------
Accurracy: 0.68
Precision: 0.68
Recall: 0.6326666666666666
F1-Score: 0.6295238095238095
---------- End eval 90/10 split -------------------



/Users/armint/PyCharmMiscProject/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/armint/PyCharmMiscProject/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/armint/PyCharmMiscProject/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

CV-5 results
accuracy: 0.6333
precision_macro: 0.6601
recall_macro: 0.6160
f1_macro: 0.5942
CV-10 results
accuracy: 0.6360
precision_macro: 0.5746
recall_macro: 0.6240
f1_macro: 0.5691


/Users/armint/PyCharmMiscProject/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Experiments

We now pick the model that we trained on the 90/10 split and analyze how hyperparameter changes affect its performance.

In [6]:
def eval_model(model, x_test, y_test, model_name):
  print(f"---------- Start eval {model_name}-------------------");
  print(f"Accuracy: {accuracy_score(model.predict(x_test), y_test)}");
  print(f"Precision: {precision_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"Recall: {recall_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"F1-Score: {f1_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"---------- End eval {model_name} -------------------\n");

# Model 1: Change split criterion from 'gini' to 'entropy'
reviews_e1_model = RandomForestClassifier(n_estimators=400, max_depth=10, max_features="sqrt", criterion="entropy", random_state=18);
reviews_e1_fit = reviews_e1_model.fit(reviews_x_train_90, reviews_y_train_90);

# Model 2: Reduce number of trees from 400 to 15
reviews_e2_model = RandomForestClassifier(n_estimators=15, max_depth=10, max_features="sqrt", criterion="gini", random_state=18);
reviews_e2_fit = reviews_e2_model.fit(reviews_x_train_90, reviews_y_train_90);

# Model 3: Increase number of trees from 400 to 800
reviews_e3_model = RandomForestClassifier(n_estimators=800, max_depth=10, max_features="sqrt", criterion="gini", random_state=18);
reviews_e3_fit = reviews_e3_model.fit(reviews_x_train_90, reviews_y_train_90);

# Model 4: Increase number of trees from 400 to 1200
reviews_e4_model = RandomForestClassifier(n_estimators=1200, max_depth=10, max_features="sqrt", criterion="gini", random_state=18);
reviews_e4_fit = reviews_e4_model.fit(reviews_x_train_90, reviews_y_train_90);

eval_model(reviews_e1_fit, reviews_x_test_90, reviews_y_test_90, "gini -> entropy");
eval_model(reviews_e2_fit, reviews_x_test_90, reviews_y_test_90, "400 trees -> 15 trees");
eval_model(reviews_e3_fit, reviews_x_test_90, reviews_y_test_90, "400 trees -> 800 trees");
eval_model(reviews_e4_fit, reviews_x_test_90, reviews_y_test_90, "400 trees -> 1200 trees");

---------- Start eval gini -> entropy-------------------
Accuracy: 0.6133333333333333
Precision: 0.6
Recall: 0.5483333333333333
F1-Score: 0.5519047619047619
---------- End eval gini -> entropy -------------------

---------- Start eval 400 trees -> 15 trees-------------------
Accuracy: 0.32
Precision: 0.32
Recall: 0.23166666666666663
F1-Score: 0.258
---------- End eval 400 trees -> 15 trees -------------------

---------- Start eval 400 trees -> 800 trees-------------------
Accuracy: 0.7466666666666667
Precision: 0.73
Recall: 0.6746666666666665
F1-Score: 0.6807619047619048
---------- End eval 400 trees -> 800 trees -------------------

---------- Start eval 400 trees -> 1200 trees-------------------
Accuracy: 0.7466666666666667
Precision: 0.74
Recall: 0.7046666666666666
F1-Score: 0.6967619047619047
---------- End eval 400 trees -> 1200 trees -------------------



# Student Dropout and Academic Success

In [7]:
!pip install ucimlrepo

In [8]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from ucimlrepo import fetch_ucirepo

# ------------------
# Load the data from the source
# ------------------

student_dataset = fetch_ucirepo(id=697);

### Preprocessing

In [9]:
# -> Since the dataset contains no missing (incl. null) values, no preprocessing in this regard is necessary
print(student_dataset.data.features.isnull().sum().sum());
print(student_dataset.data.targets.isnull().sum().sum());

# -> Since all the feature values are already numeric, no transformation is necessary

# Split into features and labels
student_dataset_features = student_dataset.data.features;
student_dataset_labels = np.array(student_dataset.data.targets).ravel();

0
0


### Data splits

We now define the three data splits used for e.g. hyperparameter tuning and holdout evaluation.

In [10]:
def create_split(features, labels, test_size, rval=18):
  return train_test_split(features, labels, test_size=test_size, random_state=rval, stratify=labels);

student_x_train_90, student_x_test_90, student_y_train_90, student_y_test_90 = create_split(student_dataset_features, student_dataset_labels, 0.1);
student_x_train_70, student_x_test_70, student_y_train_70, student_y_test_70 = create_split(student_dataset_features, student_dataset_labels, 0.3);
student_x_train_50, student_x_test_50, student_y_train_50, student_y_test_50 = create_split(student_dataset_features, student_dataset_labels, 0.5);

### Train baseline models

Use cross-validation for hyperparameter tuning.

In [11]:
student_test_hyper_params_settings = {
    "model__n_estimators": [50, 100, 150], # Nr. of decision trees
    "model__max_depth": [5, 10], # Max tree depth
    "model__max_features": ["sqrt", "log2"], # Max number of features to consider when searching for best split
    "model__criterion": ["gini", "entropy"],
};

def compute_base_line_model(x_train, y_train, cv=5):
  student_rf_pipeline = Pipeline([
    ("model", RandomForestClassifier(random_state=18)),
  ]);

  rf_baseline_grid = GridSearchCV(
    estimator=student_rf_pipeline,
    param_grid=student_test_hyper_params_settings,
    cv=cv,
    n_jobs=-1,
    scoring="accuracy",
  );

  return rf_baseline_grid.fit(x_train, y_train);

student_baseline_model_50_split = compute_base_line_model(student_x_train_50, student_y_train_50);
print(f"50/50 split - best performing hyperparameters based on accuracy: {student_baseline_model_50_split.best_params_}");
print(f"Best Score: {student_baseline_model_50_split.best_score_}\n");

student_baseline_model_70_split = compute_base_line_model(student_x_train_70, student_y_train_70);
print(f"70/30 split - best performing hyperparameters based on accuracy: {student_baseline_model_70_split.best_params_}");
print(f"Best Score: {student_baseline_model_70_split.best_score_}\n");

student_baseline_model_90_split = compute_base_line_model(student_x_train_90, student_y_train_90);
print(f"90/10 split - best performing hyperparameters based on accuracy: {student_baseline_model_90_split.best_params_}");
print(f"Best Score: {student_baseline_model_90_split.best_score_}\n");

student_baseline_model_cv = compute_base_line_model(student_dataset_features, student_dataset_labels);
print(f"CV5 - best performing hyperparameters based on accuracy: {student_baseline_model_cv.best_params_}");
print(f"Best Score: {student_baseline_model_cv.best_score_}\n");

student_baseline_model_cv10 = compute_base_line_model(student_dataset_features, student_dataset_labels, cv=10);
print(f"CV10 - best performing hyperparameters based on accuracy: {student_baseline_model_cv10.best_params_}");
print(f"Best Score: {student_baseline_model_cv10.best_score_}\n");

50/50 split - best performing hyper-parameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 150}
Best Score: 0.7631022542720857

70/30 split - best performing hyper-parameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 150}
Best Score: 0.7693751628537182

90/10 split - best performing hyper-parameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'log2', 'model__n_estimators': 150}
Best Score: 0.7729185450464368

CV5 - best performing hyper-parameters based on accuracy: {'model__criterion': 'entropy', 'model__max_depth': 10, 'model__max_features': 'log2', 'model__n_estimators': 50}
Best Score: 0.7728269039036736

CV10 - best performing hyper-parameters based on accuracy: {'model__criterion': 'entropy', 'model__max_depth': 10, 'model__max_features': 'log2', 'model__n_estimators

##### Holdout and Cross-Validation Evaluation

We now use the found hyperparameters to evaluate our models via holdout and cross-validation.

In [12]:
def student_eval_model(model, x_test, y_test, split_name):
  print(f"---------- Start eval {split_name}-------------------");
  print(f"Accuracy: {accuracy_score(model.predict(x_test), y_test)}");
  print(f"Precision: {precision_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"Recall: {recall_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"F1-Score: {f1_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"---------- End eval {split_name} -------------------\n");

# Perform holdout evaluation on the different data splits

student_eval_model(student_baseline_model_50_split, student_x_test_50, student_y_test_50, "50/50 split");
student_eval_model(student_baseline_model_70_split, student_x_test_70, student_y_test_70, "70/30 split");
student_eval_model(student_baseline_model_90_split, student_x_test_90, student_y_test_90, "90/10 split");

# Perform cross-validation evaluation with fold sizes 5 and 10

student_scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

student_scoring_cv5_pipeline = Pipeline([
  ('classifier', RandomForestClassifier(random_state=18, criterion="entropy", max_depth=10, max_features="log2", n_estimators=50)),
]);

student_scoring_cv10_pipeline = Pipeline([
  ('classifier', RandomForestClassifier(random_state=18, criterion="entropy", max_depth=10, max_features="log2", n_estimators=100)),
]);

student_cv_5_results = cross_validate(student_scoring_cv5_pipeline, student_dataset_features, student_dataset_labels, cv=5, scoring=student_scoring)
student_cv_10_results = cross_validate(student_scoring_cv10_pipeline, student_dataset_features, student_dataset_labels, cv=10, scoring=student_scoring)

print("CV-5 results");
for metric in student_scoring:
    print(f"{metric}: {student_cv_5_results['test_' + metric].mean():.4f}")

print("CV-10 results");
for metric in student_scoring:
    print(f"{metric}: {student_cv_10_results['test_' + metric].mean():.4f}")

---------- Start eval 50/50 split-------------------
Accuracy: 0.7757685352622061
Precision: 0.6752083531970937
Recall: 0.7381446706188455
F1-Score: 0.6864962420593949
---------- End eval 50/50 split -------------------

---------- Start eval 70/30 split-------------------
Accuracy: 0.7703313253012049
Precision: 0.6707329421890655
Recall: 0.7206359965708815
F1-Score: 0.6805987799960874
---------- End eval 70/30 split -------------------

---------- Start eval 90/10 split-------------------
Accuracy: 0.7832957110609481
Precision: 0.6801810485841141
Recall: 0.7285150760886484
F1-Score: 0.6871890403985549
---------- End eval 90/10 split -------------------

CV-5 results
accuracy: 0.7728
precision_macro: 0.7359
recall_macro: 0.6675
f1_macro: 0.6773
CV-10 results
accuracy: 0.7726
precision_macro: 0.7379
recall_macro: 0.6673
f1_macro: 0.6780


### Experiments

We now pick the model that we trained on the 90/10 split and analyze how hyperparameter changes affect its performance.

In [13]:
def eval_model(model, x_test, y_test, model_name):
  print(f"---------- Start eval {model_name}-------------------");
  print(f"Accuracy: {accuracy_score(model.predict(x_test), y_test)}");
  print(f"Precision: {precision_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"Recall: {recall_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"F1-Score: {f1_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"---------- End eval {model_name} -------------------\n");

# Model 1: Change split criterion from 'gini' to 'entropy'
student_e1_model = RandomForestClassifier(n_estimators=150, max_depth=10, max_features="log2", criterion="entropy", random_state=18);
student_e1_fit = student_e1_model.fit(student_x_train_90, student_y_train_90);

# Model 2: Reduce number of trees from 150 to 15
student_e2_model = RandomForestClassifier(n_estimators=15, max_depth=10, max_features="log2", criterion="gini", random_state=18);
student_e2_fit = student_e2_model.fit(student_x_train_90, student_y_train_90);

# Model 3: Increase number of trees from 150 to 400
student_e3_model = RandomForestClassifier(n_estimators=400, max_depth=10, max_features="log2", criterion="gini", random_state=18);
student_e3_fit = student_e3_model.fit(student_x_train_90, student_y_train_90);

eval_model(student_e1_fit, student_x_test_90, student_y_test_90, "gini -> entropy");
eval_model(student_e2_fit, student_x_test_90, student_y_test_90, "150 trees -> 15 trees");
eval_model(student_e3_fit, student_x_test_90, student_y_test_90, "150 trees -> 400 trees");

---------- Start eval gini -> entropy-------------------
Accuracy: 0.7742663656884876
Precision: 0.6688311239988952
Recall: 0.7058802087164917
F1-Score: 0.671943895894322
---------- End eval gini -> entropy -------------------

---------- Start eval 150 trees -> 15 trees-------------------
Accuracy: 0.7720090293453724
Precision: 0.6699811994136766
Recall: 0.7073310735282566
F1-Score: 0.6739353241489007
---------- End eval 150 trees -> 15 trees -------------------

---------- Start eval 150 trees -> 400 trees-------------------
Accuracy: 0.7787810383747178
Precision: 0.6745060862915047
Recall: 0.7128071671271874
F1-Score: 0.6795214139824682
---------- End eval 150 trees -> 400 trees -------------------



# Congressional Voting

In [14]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer

# ------------------
# Load the data from the source
# ------------------

dataset_voting = pd.read_csv("./CongressionalVotingID.shuf.lrn.csv");

### Preprocessing

In [15]:
# Remove the "ID" column as it is not needed for this task
dataset_voting = dataset_voting.drop(columns=["ID"]);

# Create working copies of the dataset for the two different missing-value strategies we are going to use
dataset_voting_third_category = dataset_voting.copy();
dataset_voting_inferred_values = dataset_voting.copy();

# 1. Strategy: Introduce a new category for missing values
vote_decision_encoding_v1 = {"y": 0, "n": 1, "unknown": 2};

for column in dataset_voting_third_category.drop(columns=["class"]):
  dataset_voting_third_category[column] = dataset_voting_third_category[column].map(vote_decision_encoding_v1);

# 2. Strategy: Impute missing values
dataset_voting_inferred_values = dataset_voting_inferred_values.replace("unknown", np.nan);
vote_decision_encoding_v2 = {"y": 0, "n": 1};

for column in dataset_voting_inferred_values.drop(columns=["class"]):
  dataset_voting_inferred_values[column] = dataset_voting_inferred_values[column].map(vote_decision_encoding_v2);

# Split features and labels

ds_voting_third_category_features = dataset_voting_third_category.drop(columns=["class"]);
ds_voting_third_category_labels = dataset_voting_third_category["class"];

ds_voting_inferred_features = dataset_voting_inferred_values.drop(columns=["class"]);
ds_voting_inferred_labels = dataset_voting_inferred_values["class"];

### Data splits

We now define the three data splits used for e.g. hyperparameter tuning and holdout evaluation.

In [16]:
def create_split(features, labels, test_size, rval=18):
  return train_test_split(features, labels, test_size=test_size, random_state=rval, stratify=labels);

# Data splits for the dataset where we introduced a third category to handle missing values
x_voting_train_90_category, x_voting_test_90_category, y_voting_train_90_category, y_voting_test_90_category = create_split(ds_voting_third_category_features, ds_voting_third_category_labels, 0.1);
x_voting_train_70_category, x_voting_test_70_category, y_voting_train_70_category, y_voting_test_70_category = create_split(ds_voting_third_category_features, ds_voting_third_category_labels, 0.3);
x_voting_train_50_category, x_voting_test_50_category, y_voting_train_50_category, y_voting_test_50_category = create_split(ds_voting_third_category_features, ds_voting_third_category_labels, 0.5);

# Data splits for the dataset where we infer a value for the missing values
x_voting_train_90_infer, x_voting_test_90_infer, y_voting_train_90_infer, y_voting_test_90_infer = create_split(ds_voting_inferred_features, ds_voting_inferred_labels, 0.1);
x_voting_train_70_infer, x_voting_test_70_infer, y_voting_train_70_infer, y_voting_test_70_infer = create_split(ds_voting_inferred_features, ds_voting_inferred_labels, 0.3);
x_voting_train_50_infer, x_voting_test_50_infer, y_voting_train_50_infer, y_voting_test_50_infer = create_split(ds_voting_inferred_features, ds_voting_inferred_labels, 0.5);

### Train baseline models

#### Missing-values Strategy 1 (Third category)

Use cross-validation for hyperparameter tuning.

In [17]:
voting_category_test_hyper_params_settings = {
    "model__n_estimators": [50, 100, 150], # Nr. of decision trees
    "model__max_depth": [5, 10], # Max tree depth
    "model__max_features": ["sqrt", "log2"], # Max number of features to consider when searching for best split
    "model__criterion": ["gini", "entropy"],
};

def compute_base_line_model(x_train, y_train, cv=5):
  voting_category_rf_pipeline = Pipeline([
    ("model", RandomForestClassifier(random_state=18)),
  ]);

  rf_baseline_grid = GridSearchCV(
    estimator=voting_category_rf_pipeline,
    param_grid=voting_category_test_hyper_params_settings,
    cv=cv,
    n_jobs=-1,
    scoring="accuracy",
  );

  return rf_baseline_grid.fit(x_train, y_train);

voting_baseline_model_50_split_category = compute_base_line_model(x_voting_train_50_category, y_voting_train_50_category);
print(f"50/50 split - best performing hyperparameters based on accuracy: {voting_baseline_model_50_split_category.best_params_}");
print(f"Best Score: {voting_baseline_model_50_split_category.best_score_}\n");

voting_baseline_model_70_split_category = compute_base_line_model(x_voting_train_70_category, y_voting_train_70_category);
print(f"70/30 split - best performing hyperparameters based on accuracy: {voting_baseline_model_70_split_category.best_params_}");
print(f"Best Score: {voting_baseline_model_70_split_category.best_score_}\n");

voting_baseline_model_90_split_category = compute_base_line_model(x_voting_train_90_category, y_voting_train_90_category);
print(f"90/10 split - best performing hyperparameters based on accuracy: {voting_baseline_model_90_split_category.best_params_}");
print(f"Best Score: {voting_baseline_model_90_split_category.best_score_}\n");

voting_baseline_model_category_cv5 = compute_base_line_model(ds_voting_third_category_features, ds_voting_third_category_labels);
print(f"CV5 - best performing hyperparameters based on accuracy: {voting_baseline_model_category_cv5.best_params_}");
print(f"Best Score: {voting_baseline_model_category_cv5.best_score_}\n");

voting_baseline_model_category_cv10 = compute_base_line_model(ds_voting_third_category_features, ds_voting_third_category_labels, cv=10);
print(f"CV10 - best performing hyperparameters based on accuracy: {voting_baseline_model_category_cv10.best_params_}");
print(f"Best Score: {voting_baseline_model_category_cv10.best_score_}\n");

50/50 split - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 5, 'model__max_features': 'sqrt', 'model__n_estimators': 150}
Best Score: 0.9445887445887446

70/30 split - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 100}
Best Score: 0.9408602150537634

90/10 split - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 50}
Best Score: 0.9592307692307692

CV5 - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 5, 'model__max_features': 'sqrt', 'model__n_estimators': 100}
Best Score: 0.9586680761099367

CV10 - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 100}
Best 

##### Holdout Evaluation

We now use the found hyperparameters to evaluate our models via the holdout method.

In [5]:
def eval_model(model, x_test, y_test, split_name):
  print(f"---------- Start eval {split_name}-------------------");
  print(f"Accuracy: {accuracy_score(model.predict(x_test), y_test)}");
  print(f"Precision: {precision_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"Recall: {recall_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"F1-Score: {f1_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"---------- End eval {split_name} -------------------\n");

# Perform holdout evaluation on our three data splits

eval_model(voting_baseline_model_50_split_category, x_voting_test_50_category, y_voting_test_50_category, "50/50 split");
eval_model(voting_baseline_model_70_split_category, x_voting_test_70_category, y_voting_test_70_category, "70/30 split");
eval_model(voting_baseline_model_90_split_category, x_voting_test_90_category, y_voting_test_90_category, "90/10 split");

---------- Start eval 50/50 split-------------------
Accurracy: 0.963302752293578
Precision: 0.9588541666666667
Recall: 0.9656448202959831
F1-Score: 0.961888111888112
---------- End eval 50/50 split -------------------

---------- Start eval 70/30 split-------------------
Accurracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
---------- End eval 70/30 split -------------------

---------- Start eval 90/10 split-------------------
Accurracy: 0.9545454545454546
Precision: 0.9444444444444444
Recall: 0.9642857142857143
F1-Score: 0.9520697167755992
---------- End eval 90/10 split -------------------



#### Missing-values Strategy 2 (Impute values)

Use cross-validation for hyperparameter tuning.

In [18]:
voting_infer_test_hyper_params_settings = {
    "model__n_estimators": [50, 100, 150], # Nr. of decision trees
    "model__max_depth": [5, 10], # Max tree depth
    "model__max_features": ["sqrt", "log2"], # Max number of features to consider when searching for best split
    "model__criterion": ["gini", "entropy"],
};

def compute_base_line_model(x_train, y_train, cv=5):
  voting_infer_rf_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("model", RandomForestClassifier(random_state=18)),
  ]);

  rf_baseline_grid = GridSearchCV(
    estimator=voting_infer_rf_pipeline,
    param_grid=voting_infer_test_hyper_params_settings,
    cv=cv,
    n_jobs=-1,
    scoring="accuracy",
  );

  return rf_baseline_grid.fit(x_train, y_train);

voting_baseline_model_50_split_infer = compute_base_line_model(x_voting_train_50_infer, y_voting_train_50_infer);
print(f"50/50 split - best performing hyperparameters based on accuracy: {voting_baseline_model_50_split_infer.best_params_}");
print(f"Best Score: {voting_baseline_model_50_split_infer.best_score_}\n");

voting_baseline_model_70_split_infer = compute_base_line_model(x_voting_train_70_infer, y_voting_train_70_infer);
print(f"70/30 split - best performing hyperparameters based on accuracy: {voting_baseline_model_70_split_infer.best_params_}");
print(f"Best Score: {voting_baseline_model_70_split_infer.best_score_}\n");

voting_baseline_model_90_split_infer = compute_base_line_model(x_voting_train_90_infer, y_voting_train_90_infer);
print(f"90/10 split - best performing hyperparameters based on accuracy: {voting_baseline_model_90_split_infer.best_params_}");
print(f"Best Score: {voting_baseline_model_90_split_infer.best_score_}\n");

voting_baseline_model_infer_cv5 = compute_base_line_model(ds_voting_inferred_features, ds_voting_inferred_labels);
print(f"CV5 - best performing hyperparameters based on accuracy: {voting_baseline_model_infer_cv5.best_params_}");
print(f"Best Score: {voting_baseline_model_infer_cv5.best_score_}\n");

voting_baseline_model_infer_cv10 = compute_base_line_model(ds_voting_inferred_features, ds_voting_inferred_labels, cv=10);
print(f"CV10 - best performing hyperparameters based on accuracy: {voting_baseline_model_infer_cv10.best_params_}");
print(f"Best Score: {voting_baseline_model_infer_cv10.best_score_}\n");

50/50 split - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 5, 'model__max_features': 'sqrt', 'model__n_estimators': 50}
Best Score: 0.9541125541125541

70/30 split - best performing hyperparameters based on accuracy: {'model__criterion': 'entropy', 'model__max_depth': 5, 'model__max_features': 'sqrt', 'model__n_estimators': 100}
Best Score: 0.9408602150537634

90/10 split - best performing hyperparameters based on accuracy: {'model__criterion': 'entropy', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 50}
Best Score: 0.9643589743589743

CV5 - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 50}
Best Score: 0.9678646934460888

CV10 - best performing hyperparameters based on accuracy: {'model__criterion': 'entropy', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 50}

##### Holdout and Cross-validation evaluation

We now use the found hyperparameters to evaluate our models via holdout and cross-validation.

In [19]:
def eval_model(model, x_test, y_test, split_name):
  print(f"---------- Start eval {split_name}-------------------");
  print(f"Accuracy: {accuracy_score(model.predict(x_test), y_test)}");
  print(f"Precision: {precision_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"Recall: {recall_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"F1-Score: {f1_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"---------- End eval {split_name} -------------------\n");

# Perform holdout evaluation on the different data splits

eval_model(voting_baseline_model_50_split_infer, x_voting_test_50_infer, y_voting_test_50_infer, "50/50 split");
eval_model(voting_baseline_model_70_split_infer, x_voting_test_70_infer, y_voting_test_70_infer, "70/30 split");
eval_model(voting_baseline_model_90_split_infer, x_voting_test_90_infer, y_voting_test_90_infer, "90/10 split");

# Perform cross-validation evaluation with fold sizes 5 and 10

voting_infer_scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

voting_infer_scoring_cv5_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy="most_frequent")),
  ('classifier', RandomForestClassifier(random_state=18, criterion="gini", max_depth=10, max_features="sqrt", n_estimators=50)),
]);

voting_infer_scoring_cv10_pipeline = Pipeline([
  ('imputer', SimpleImputer(strategy="most_frequent")),
  ('classifier', RandomForestClassifier(random_state=18, criterion="entropy", max_depth=10, max_features="sqrt", n_estimators=50)),
]);

voting_infer_cv_5_results = cross_validate(voting_infer_scoring_cv5_pipeline, ds_voting_inferred_features, ds_voting_inferred_labels, cv=5, scoring=voting_infer_scoring)
voting_infer_cv_10_results = cross_validate(voting_infer_scoring_cv10_pipeline, ds_voting_inferred_features, ds_voting_inferred_labels, cv=10, scoring=voting_infer_scoring)

print("CV-5 results");
for metric in voting_infer_scoring:
    print(f"{metric}: {voting_infer_cv_5_results['test_' + metric].mean():.4f}")

print("CV-10 results");
for metric in voting_infer_scoring:
    print(f"{metric}: {voting_infer_cv_10_results['test_' + metric].mean():.4f}")

---------- Start eval 50/50 split-------------------
Accuracy: 0.9724770642201835
Precision: 0.9699652777777779
Recall: 0.9732517482517482
F1-Score: 0.9715181604389862
---------- End eval 50/50 split -------------------

---------- Start eval 70/30 split-------------------
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
---------- End eval 70/30 split -------------------

---------- Start eval 90/10 split-------------------
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
---------- End eval 90/10 split -------------------

CV-5 results
accuracy: 0.9679
precision_macro: 0.9676
recall_macro: 0.9678
f1_macro: 0.9669
CV-10 results
accuracy: 0.9680
precision_macro: 0.9685
recall_macro: 0.9673
f1_macro: 0.9668


### Experiments

We now pick the model that we trained on the 70/30 split and analyze how hyperparameter changes affect its performance.


In [20]:
def eval_model(model, x_test, y_test, model_name):
  print(f"---------- Start eval {model_name}-------------------");
  print(f"Accuracy: {accuracy_score(model.predict(x_test), y_test)}");
  print(f"Precision: {precision_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"Recall: {recall_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"F1-Score: {f1_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"---------- End eval {model_name} -------------------\n");

# Model 1: Change split criterion from 'entropy' to 'gini'
e1_model = RandomForestClassifier(n_estimators=100, max_depth=5, max_features="sqrt", criterion="gini", random_state=18);
e1_fit = e1_model.fit(x_voting_train_70_infer, y_voting_train_70_infer);

# Model 2: Reduce number of trees from 100 to 5
e2_model = RandomForestClassifier(n_estimators=5, max_depth=5, max_features="sqrt", criterion="entropy", random_state=18);
e2_fit = e2_model.fit(x_voting_train_70_infer, y_voting_train_70_infer);

# Model 3: Reduce number of trees from 100 to 10
e3_model = RandomForestClassifier(n_estimators=10, max_depth=5, max_features="sqrt", criterion="entropy", random_state=18);
e3_fit = e3_model.fit(x_voting_train_70_infer, y_voting_train_70_infer);

# Model 4: Increase number of trees from 100 to 400
e4_model = RandomForestClassifier(n_estimators=400, max_depth=5, max_features="sqrt", criterion="entropy", random_state=18);
e4_fit = e4_model.fit(x_voting_train_70_infer, y_voting_train_70_infer);

# Model 5: Increase number of trees from 100 to 600
e5_model = RandomForestClassifier(n_estimators=600, max_depth=5, max_features="sqrt", criterion="entropy", random_state=18);
e5_fit = e5_model.fit(x_voting_train_70_infer, y_voting_train_70_infer);

eval_model(e1_fit, x_voting_test_70_infer, y_voting_test_70_infer, "entropy -> gini");
eval_model(e2_fit, x_voting_test_70_infer, y_voting_test_70_infer, "100 trees -> 5 trees");
eval_model(e3_fit, x_voting_test_70_infer, y_voting_test_70_infer, "100 trees -> 10 trees");
eval_model(e4_fit, x_voting_test_70_infer, y_voting_test_70_infer, "100 trees -> 400 trees");
eval_model(e5_fit, x_voting_test_70_infer, y_voting_test_70_infer, "100 trees -> 600 trees");

---------- Start eval entropy -> gini-------------------
Accuracy: 0.9848484848484849
Precision: 0.986842105263158
Recall: 0.9827586206896552
F1-Score: 0.984561403508772
---------- End eval entropy -> gini -------------------

---------- Start eval 100 trees -> 5 trees-------------------
Accuracy: 0.9848484848484849
Precision: 0.9821428571428572
Recall: 0.9871794871794872
F1-Score: 0.9844155844155844
---------- End eval 100 trees -> 5 trees -------------------

---------- Start eval 100 trees -> 10 trees-------------------
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Score: 1.0
---------- End eval 100 trees -> 10 trees -------------------

---------- Start eval 100 trees -> 400 trees-------------------
Accuracy: 0.9848484848484849
Precision: 0.986842105263158
Recall: 0.9827586206896552
F1-Score: 0.984561403508772
---------- End eval 100 trees -> 400 trees -------------------

---------- Start eval 100 trees -> 600 trees-------------------
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-Scor

# Airplane Passenger Satisfaction

In [21]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# ------------------
# Load the data from the source
# ------------------

airplane_dataset = pd.read_csv("./combined.csv");

### Preprocessing

In [22]:
# Remove the not-needed ID column
airplane_dataset.drop(columns=["id"], inplace=True);

# Since the sklearn RF implementation can only handle numerical data, we use
#   the LabelEncoder to encode the features with categorical values
for col in ['Gender', 'Customer Type', 'Type of Travel', 'Class']:
  airplane_dataset[col] = LabelEncoder().fit_transform(airplane_dataset[col]);

# Make copies for our two different strategies of dealing with missing values
airplane_dataset_del = airplane_dataset.copy();
airplane_dataset_infer = airplane_dataset.copy();

# Strategy 1: Delete instances with missing values from the training set
airplane_dataset_del.dropna(inplace=True);
print(f"Mising values in training data {airplane_dataset_del.isnull().sum().sum()}");

# Split features and labels
airplane_dataset_del_features = airplane_dataset_del.drop(columns=["satisfaction"]);
airplane_dataset_del_labels = airplane_dataset_del["satisfaction"];

airplane_dataset_infer_features = airplane_dataset_infer.drop(columns=["satisfaction"]);
airplane_dataset_infer_labels = airplane_dataset_infer["satisfaction"];

Mising values in training data 0


### Data splits

We now define the three data splits used for e.g. hyperparameter tuning and holdout evaluation.

In [23]:
def create_split(features, labels, test_size, rval=18):
  return train_test_split(features, labels, test_size=test_size, random_state=rval, stratify=labels);

# Data splits for the dataset where we introduced a third category to handle missing values
x_plane_train_90_del, x_plane_test_90_del, y_plane_train_90_del, y_plane_test_90_del = create_split(airplane_dataset_del_features, airplane_dataset_del_labels, 0.1);
x_plane_train_70_del, x_plane_test_70_del, y_plane_train_70_del, y_plane_test_70_del = create_split(airplane_dataset_del_features, airplane_dataset_del_labels, 0.3);
x_plane_train_50_del, x_plane_test_50_del, y_plane_train_50_del, y_plane_test_50_del = create_split(airplane_dataset_del_features, airplane_dataset_del_labels, 0.5);

# Data splits for the dataset where we infer a value for the missing values
x_plane_train_90_infer, x_plane_test_90_infer, y_plane_train_90_infer, y_plane_test_90_infer = create_split(airplane_dataset_infer_features, airplane_dataset_infer_labels, 0.1);
x_plane_train_70_infer, x_plane_test_70_infer, y_plane_train_70_infer, y_plane_test_70_infer = create_split(airplane_dataset_infer_features, airplane_dataset_infer_labels, 0.3);
x_plane_train_50_infer, x_plane_test_50_infer, y_plane_train_50_infer, y_plane_test_50_infer = create_split(airplane_dataset_infer_features, airplane_dataset_infer_labels, 0.5);

### Train baseline models

#### Missing Values Strategy 1 (Delete instances w/ missing values)

Use cross-validation for hyperparameter tuning.

In [24]:
plane_del_test_hyper_params_settings = {
    "model__n_estimators": [50, 100, 200], # Nr. of decision trees
    "model__max_depth": [5, 10], # Max tree depth
    "model__max_features": ["sqrt", "log2"], # Max number of features to consider when searching for best split
    "model__criterion": ["gini", "entropy"],
};

def compute_base_line_model(x_train, y_train, cv=5):
  plane_del_rf_pipeline = Pipeline([
    ("model", RandomForestClassifier(random_state=18)),
  ]);

  rf_baseline_grid = GridSearchCV(
    estimator=plane_del_rf_pipeline,
    param_grid=plane_del_test_hyper_params_settings,
    cv=cv,
    n_jobs=-1,
    scoring="accuracy",
  );

  return rf_baseline_grid.fit(x_train, y_train);

plane_del_baseline_model_50_split = compute_base_line_model(x_plane_train_50_del, y_plane_train_50_del);
print(f"50/50 split - best performing hyperparameters based on accuracy: {plane_del_baseline_model_50_split.best_params_}");
print(f"Best Score: {plane_del_baseline_model_50_split.best_score_}\n");

plane_del_baseline_model_70_split = compute_base_line_model(x_plane_train_70_del, y_plane_train_70_del);
print(f"70/30 split - best performing hyperparameters based on accuracy: {plane_del_baseline_model_70_split.best_params_}");
print(f"Best Score: {plane_del_baseline_model_70_split.best_score_}\n");

plane_del_baseline_model_90_split = compute_base_line_model(x_plane_train_90_del, y_plane_train_90_del);
print(f"90/10 split - best performing hyperparameters based on accuracy: {plane_del_baseline_model_90_split.best_params_}");
print(f"Best Score: {plane_del_baseline_model_90_split.best_score_}\n");

plane_del_baseline_model_cv = compute_base_line_model(airplane_dataset_del_features, airplane_dataset_del_labels);
print(f"CV5 - best performing hyperparameters based on accuracy: {plane_del_baseline_model_cv.best_params_}");
print(f"Best Score: {plane_del_baseline_model_cv.best_score_}\n");

plane_del_baseline_model_cv10 = compute_base_line_model(airplane_dataset_del_features, airplane_dataset_del_labels, cv=10);
print(f"CV10 - best performing hyperparameters based on accuracy: {plane_del_baseline_model_cv10.best_params_}");
print(f"Best Score: {plane_del_baseline_model_cv10.best_score_}\n");

50/50 split - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 200}
Best Score: 0.9455230737786863

70/30 split - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 200}
Best Score: 0.9459068843777582

90/10 split - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 50}
Best Score: 0.9457601901876233

CV5 - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 200}
Best Score: 0.9463189529471728

CV10 - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 50}
Best

##### Holdout and Cross-Validation Evaluation

We now use the found hyperparameters to evaluate our models via holdout and cross-validation.

In [26]:
def eval_model(model, x_test, y_test, split_name):
  print(f"---------- Start eval {split_name}-------------------");
  print(f"Accurracy: {accuracy_score(model.predict(x_test), y_test)}");
  print(f"Precision: {precision_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"Recall: {recall_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"F1-Score: {f1_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"---------- End eval {split_name} -------------------\n");

# Perform holdout evaluation on our different data splits

eval_model(plane_del_baseline_model_50_split, x_plane_test_50_del, y_plane_test_50_del, "50/50 split");
eval_model(plane_del_baseline_model_70_split, x_plane_test_70_del, y_plane_test_70_del, "70/30 split");
eval_model(plane_del_baseline_model_90_split, x_plane_test_90_del, y_plane_test_90_del, "90/10 split");

# Perform cross-validation for fold sizes 5 and 10

plane_del_scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

plane_del_scoring_cv5_pipeline = Pipeline([
  ('classifier', RandomForestClassifier(random_state=18, criterion="gini", max_depth=10, max_features="sqrt", n_estimators=200)),
]);

plane_del_scoring_cv10_pipeline = Pipeline([
  ('classifier', RandomForestClassifier(random_state=18, criterion="gini", max_depth=10, max_features="sqrt", n_estimators=50)),
]);

plane_del_cv_5_results = cross_validate(plane_del_scoring_cv5_pipeline, airplane_dataset_del_features, airplane_dataset_del_labels, cv=5, scoring=plane_del_scoring)
plane_del_cv_10_results = cross_validate(plane_del_scoring_cv10_pipeline, airplane_dataset_del_features, airplane_dataset_del_labels, cv=10, scoring=plane_del_scoring)

print("CV-5 results");
for metric in plane_del_scoring:
    print(f"{metric}: {plane_del_cv_5_results['test_' + metric].mean():.4f}")

print("CV-10 results");
for metric in plane_del_scoring:
    print(f"{metric}: {plane_del_cv_10_results['test_' + metric].mean():.4f}")

---------- Start eval 50/50 split-------------------
Accurracy: 0.946682318052638
Precision: 0.944672417146547
Recall: 0.9467192264761582
F1-Score: 0.9456303035205267
---------- End eval 50/50 split -------------------

---------- Start eval 70/30 split-------------------
Accurracy: 0.9460447396195331
Precision: 0.944073295544695
Recall: 0.9460179594424171
F1-Score: 0.9449860361564144
---------- End eval 70/30 split -------------------

---------- Start eval 90/10 split-------------------
Accurracy: 0.9489535871495869
Precision: 0.9470009100662591
Recall: 0.9490215463586182
F1-Score: 0.9479477868798809
---------- End eval 90/10 split -------------------

CV-5 results
accuracy: 0.9463
precision_macro: 0.9465
recall_macro: 0.9442
f1_macro: 0.9452
CV-10 results
accuracy: 0.9465
precision_macro: 0.9467
recall_macro: 0.9443
f1_macro: 0.9454


#### Missing Values Strategy 2 (Impute missing values)

Use cross-validation for hyperparameter tuning.

In [27]:
plane_infer_test_hyper_params_settings = {
    "model__n_estimators": [50, 100, 200], # Nr. of decision trees
    "model__max_depth": [5, 10], # Max tree depth
    "model__max_features": ["sqrt", "log2"], # Max number of features to consider when searching for best split
    "model__criterion": ["gini", "entropy"],
};

def compute_base_line_model(x_train, y_train, cv=5):
  plane_infer_rf_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("model", RandomForestClassifier(random_state=18)),
  ]);

  rf_baseline_grid = GridSearchCV(
    estimator=plane_infer_rf_pipeline,
    param_grid=plane_infer_test_hyper_params_settings,
    cv=cv,
    n_jobs=-1,
    scoring="accuracy",
  );

  return rf_baseline_grid.fit(x_train, y_train);

plane_infer_baseline_model_50_split = compute_base_line_model(x_plane_train_50_infer, y_plane_train_50_infer);
print(f"50/50 split - best performing hyperparameters based on accuracy: {plane_infer_baseline_model_50_split.best_params_}");
print(f"Best Score: {plane_infer_baseline_model_50_split.best_score_}\n");

plane_infer_baseline_model_70_split = compute_base_line_model(x_plane_train_70_infer, y_plane_train_70_infer);
print(f"70/30 split - best performing hyperparameters based on accuracy: {plane_infer_baseline_model_70_split.best_params_}");
print(f"Best Score: {plane_infer_baseline_model_70_split.best_score_}\n");

plane_infer_baseline_model_90_split = compute_base_line_model(x_plane_train_90_infer, y_plane_train_90_infer);
print(f"90/10 split - best performing hyperparameters based on accuracy: {plane_infer_baseline_model_90_split.best_params_}");
print(f"Best Score: {plane_infer_baseline_model_90_split.best_score_}\n");

plane_infer_baseline_model_cv5 = compute_base_line_model(airplane_dataset_infer_features, airplane_dataset_infer_labels);
print(f"CV 5 - best performing hyperparameters based on accuracy: {plane_infer_baseline_model_cv5.best_params_}");
print(f"Best Score: {plane_infer_baseline_model_cv5.best_score_}\n");

plane_infer_baseline_model_cv10 = compute_base_line_model(airplane_dataset_infer_features, airplane_dataset_infer_labels, cv=10);
print(f"CV 10 - best performing hyperparameters based on accuracy: {plane_infer_baseline_model_cv10.best_params_}");
print(f"Best Score: {plane_infer_baseline_model_cv10.best_score_}\n");

50/50 split - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 50}
Best Score: 0.9462272867262088

70/30 split - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 200}
Best Score: 0.9464560647156703

90/10 split - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 200}
Best Score: 0.9460441832580567

CV 5 - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 200}
Best Score: 0.9459424083769633

CV 10 - best performing hyperparameters based on accuracy: {'model__criterion': 'gini', 'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__n_estimators': 200}
B

##### Holdout Evaluation

We now use the found hyperparameters to evaluate our models via the holdout method.

In [28]:
def eval_model(model, x_test, y_test, split_name):
  print(f"---------- Start eval {split_name}-------------------");
  print(f"Accuracy: {accuracy_score(model.predict(x_test), y_test)}");
  print(f"Precision: {precision_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"Recall: {recall_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"F1-Score: {f1_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"---------- End eval {split_name} -------------------\n");

# Perform holdout evaluation on our three data splits

eval_model(plane_infer_baseline_model_50_split, x_plane_test_50_infer, y_plane_test_50_infer, "50/50 split");
eval_model(plane_infer_baseline_model_70_split, x_plane_test_70_infer, y_plane_test_70_infer, "70/30 split");
eval_model(plane_infer_baseline_model_90_split, x_plane_test_90_infer, y_plane_test_90_infer, "90/10 split");

---------- Start eval 50/50 split-------------------
Accuracy: 0.9443640283338466
Precision: 0.9423132754975383
Recall: 0.9443478254302151
F1-Score: 0.9432653453157476
---------- End eval 50/50 split -------------------

---------- Start eval 70/30 split-------------------
Accuracy: 0.9446155425520993
Precision: 0.942400958381902
Recall: 0.9447758855262705
F1-Score: 0.943501237646475
---------- End eval 70/30 split -------------------

---------- Start eval 90/10 split-------------------
Accuracy: 0.945642131198029
Precision: 0.9432140665489523
Recall: 0.9460749500552559
F1-Score: 0.9445213924542839
---------- End eval 90/10 split -------------------



### Experiments

We now pick the model that we trained on the 90/10 split and analyze how hyperparameter changes affect its performance.

In [29]:
def eval_model(model, x_test, y_test, model_name):
  print(f"---------- Start eval {model_name}-------------------");
  print(f"Accuracy: {accuracy_score(model.predict(x_test), y_test)}");
  print(f"Precision: {precision_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"Recall: {recall_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"F1-Score: {f1_score(model.predict(x_test), y_test, average='macro', zero_division=0)}");
  print(f"---------- End eval {model_name} -------------------\n");

# Model 1: Change split criterion from 'gini' to 'entropy'
plane_e1_model = RandomForestClassifier(n_estimators=50, max_depth=10, max_features="sqrt", criterion="entropy", random_state=18);
plane_e1_fit = plane_e1_model.fit(x_plane_train_90_del, y_plane_train_90_del);

# Model 2: Reduce number of trees from 100 to 5
plane_e2_model = RandomForestClassifier(n_estimators=5, max_depth=10, max_features="sqrt", criterion="gini", random_state=18);
plane_e2_fit = plane_e2_model.fit(x_plane_train_90_del, y_plane_train_90_del);

# Model 3: Increase number of trees from 100 to 300
plane_e3_model = RandomForestClassifier(n_estimators=300, max_depth=10, max_features="sqrt", criterion="gini", random_state=18);
plane_e3_fit = plane_e3_model.fit(x_plane_train_90_del, y_plane_train_90_del);

eval_model(plane_e1_fit, x_plane_test_90_del, y_plane_test_90_del, "gini -> entropy");
eval_model(plane_e2_fit, x_plane_test_90_del, y_plane_test_90_del, "50 trees -> 5 trees");
eval_model(plane_e3_fit, x_plane_test_90_del, y_plane_test_90_del, "50 trees -> 300 trees");

---------- Start eval gini -> entropy-------------------
Accuracy: 0.9480268746621361
Precision: 0.9460785984232023
Recall: 0.9480535306795133
F1-Score: 0.9470051339181861
---------- End eval gini -> entropy -------------------

---------- Start eval 50 trees -> 5 trees-------------------
Accuracy: 0.9416943393312225
Precision: 0.93961480755038
Recall: 0.9416020554838642
F1-Score: 0.9405455054376853
---------- End eval 50 trees -> 5 trees -------------------

---------- Start eval 50 trees -> 300 trees-------------------
Accuracy: 0.9497258475557958
Precision: 0.9479102360196429
Recall: 0.9496861817372781
F1-Score: 0.9487486832490173
---------- End eval 50 trees -> 300 trees -------------------

